In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/pakistans-largest-ecommerce-dataset/Pakistan Largest Ecommerce Dataset.csv')

data

In [ ]:
print(data.shape) # shape of data
print(data.columns) # list of columns

# Subset of data needed for Payment Methods versus Order Status analysis

In [ ]:
data_sub = data[['payment_method', 'status', 'grand_total', 'Year', 'Month']]
data_sub

# Treatment of Null or NaN

In [ ]:
# there are some rows in which all columns are null need to drop all those rows
data_sub.dropna(how='all', inplace=True)

#null_status_rows = pd.isnull(data_sub.status)
data_sub.sort_index(ascending=False)
print(data_sub.shape)

# Status null treatment
> After removing all rows having nan in all column, only 15 are rows in which status is nullwe use mode to all 15 nulls. Mode value is completed.

In [ ]:
data_sub.status.fillna(data_sub.status.mode()[0], inplace=True)
print(data_sub.isnull().sum())

# Updating some payment type methods 
> Replacing *cashatdoorstep* with *cod* and *Easypay_MA* with *Easypay*

In [ ]:
data_sub['payment_method'].replace({'cashatdoorstep': 'cod', 'Easypay_MA': 'Easypay'}, inplace=True)

# Types of Payment Methods And List of statuses

In [ ]:
type_of_payment_methods = data_sub.payment_method.unique()
print(type_of_payment_methods)
print(f'Total number of Payment Method {len(type_of_payment_methods)}')
list_of_statues = data_sub.status.unique()
print(list_of_statues)

# Number of orders by Payment Methods

In [ ]:
number_of_orders_by_payments_methods = data_sub.groupby('payment_method').size().reset_index(name='Number of orders').sort_values('Number of orders', ascending=False)
print(number_of_orders_by_payments_methods)
bar = px.bar(number_of_orders_by_payments_methods, y='Number of orders', x='payment_method',
             title='Number of orders by Payment method',
             hover_data=['payment_method'], labels={'payment_method':'Payment Method'})
bar.show()

# Best Payment Method W.r.t completion of orders

> Considering order statuses 'complete', 'paid', 'received' as completed

In [ ]:
completed_orders = data_sub[(data_sub['status'] == 'complete') | (data_sub['status'] == 'paid') | (data_sub['status'] == 'received')]
completed_orders_by_payment_methods = completed_orders.groupby('payment_method').size().reset_index(name='Number of orders').sort_values('Number of orders', ascending=False)

fig = px.bar(completed_orders_by_payment_methods, y='Number of orders', x='payment_method',
             title='Number of completed orders by Paymend Methods',
             hover_data=['payment_method'], labels={'payment_method':'Payment method'})
fig.show()

# Worst Payment Method W.r.t completion of orders

In [ ]:
not_completed_orders = data_sub[(data_sub['status'] != 'complete') & (data_sub['status'] != 'paid') & (data_sub['status'] != 'received')]
not_completed_orders_by_payment_methods = not_completed_orders.groupby('payment_method').size().reset_index(name='Number of orders').sort_values('Number of orders', ascending=False)

fig = px.bar(not_completed_orders_by_payment_methods, y='Number of orders', x='payment_method',
             title='Number of not completed orders by Paymend Methods',
             hover_data=['payment_method'], labels={'payment_method':'Payment method'})
fig.update_traces(marker_color='red')
fig.show()

# Compare the completed and not completed orders by Payment Methods

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='Completed', x=completed_orders_by_payment_methods['payment_method'], y=completed_orders_by_payment_methods['Number of orders']),
    go.Bar(name='Not Completed', x=not_completed_orders_by_payment_methods['payment_method'], y=not_completed_orders_by_payment_methods['Number of orders'])
])
fig.update_layout(barmode='group')
fig.show()

# Payment methods by number of completed order Per Year

In [ ]:
completed_orders_by_payment_methods_year = completed_orders.groupby(['payment_method', 'Year']).size().reset_index(name='Number of orders').sort_values(['Number of orders', 'Year'], ascending=False)
bar = px.bar(completed_orders_by_payment_methods_year, y='Number of orders', x='payment_method',
             title='Number of completed orders by Category Per Year', color='Year',
             hover_data=['payment_method'], labels={'payment_method':'Payment Method'})
bar.show()

# Please Upvote if you find the notebook interesting
> Work in progress